In [ ]:
!pip install pycryptodome
!pip install numpy
!pip install cryptography

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
from Crypto.Random import get_random_bytes
from ccakem import (
    kem_keygen512, kem_keygen768, kem_keygen1024,
    kem_encaps512, kem_encaps768, kem_encaps1024,
    kem_decaps512, kem_decaps768, kem_decaps1024
)
from cpake import decrypt, encrypt  # Importando as funções necessárias para encriptar e descriptografar
from params import (
    KYBER_512SK_BYTES, KYBER_768SK_BYTES, KYBER_1024SK_BYTES, KYBER_SYM_BYTES,
    KYBER_INDCPA_SECRETKEY_BYTES_K512, KYBER_INDCPA_PUBLICKEYBYTES_K512,
    KYBER_INDCPA_SECRETKEY_BYTES_K768, KYBER_INDCPA_PUBLICKEYBYTES_K768,
    KYBER_INDCPA_SECRETKEY_BYTES_K1024, KYBER_INDCPA_PUBLICKEYBYTES_K1024,
    KYBER_SS_BYTES  # Agora incluído o KYBER_SS_BYTES do arquivo params.py
)
import binascii

# Função para converter um valor em byte
def cast_to_byte(x):
    """Converte um valor inteiro para byte (0 a 255)."""
    return x & 0xFF

# Função para exibir menu de escolha de ação (Criptografar ou Descriptografar)
def escolher_acao():
    print("Escolha uma ação:")
    print("1 - Criptografar")
    print("2 - Descriptografar")

    opcao = input("Digite o número da sua escolha: ")

    if opcao == '1':
        return 'criptografar'
    elif opcao == '2':
        return 'descriptografar'
    else:
        print("Opção inválida.")
        return escolher_acao()

# Função para exibir menu de escolha de parâmetros Kyber
def escolher_parametro():
    print("Escolha um dos parâmetros Kyber:")
    print("1 - Kyber-512")
    print("2 - Kyber-512-90s")
    print("3 - Kyber-768")
    print("4 - Kyber-768-90s")
    print("5 - Kyber-1024")
    print("6 - Kyber-1024-90s")

    opcao = input("Digite o número da sua escolha: ")

    if opcao == '1':
        return 512
    elif opcao == '2':
        return '512-90s'
    elif opcao == '3':
        return 768
    elif opcao == '4':
        return '768-90s'
    elif opcao == '5':
        return 1024
    elif opcao == '6':
        return '1024-90s'
    else:
        print("Opção inválida.")
        return escolher_parametro()

# Função para gerar chave e realizar encapsulamento com o parâmetro escolhido
def encriptar_parametro(parametro, mensagem=None):
    if parametro == 512:
        sk, pk = kem_keygen512()
        print("Chave gerada para Kyber-512")
        shared_secret, ciphertext = kem_encaps512(pk)
    elif parametro == 768:
        sk, pk = kem_keygen768()
        print("Chave gerada para Kyber-768")
        shared_secret, ciphertext = kem_encaps768(pk)
    elif parametro == 1024:
        sk, pk = kem_keygen1024()
        print("Chave gerada para Kyber-1024")
        shared_secret, ciphertext = kem_encaps1024(pk)
    else:
        print("Somente os modos normais de Kyber foram implementados no momento.")
        return

    print("Chave compartilhada:", shared_secret)
    print("Ciphertext:", ciphertext)

    return sk, ciphertext

# Função para realizar descriptografia com base no parâmetro escolhido
def descriptografar_parametro(parametro, sk, ciphertext):
    if parametro == 512:
        shared_secret_dec = kem_decaps512(sk, ciphertext)
    elif parametro == 768:
        shared_secret_dec = kem_decaps768(sk, ciphertext)
    elif parametro == 1024:
        shared_secret_dec = kem_decaps1024(sk, ciphertext)
    else:
        print("Somente os modos normais de Kyber foram implementados no momento.")
        return

    print("Chave compartilhada decapsulada:", shared_secret_dec)
    return shared_secret_dec

# Função para converter lista de inteiros em bytes
def lista_para_bytes(lista):
    return bytes([x & 0xFF for x in lista])

# Função principal para escolher a ação e o modo Kyber e realizar o processo
def main():
    acao = escolher_acao()

    if acao == 'criptografar':
        parametro_escolhido = escolher_parametro()
        mensagem = input("Digite a mensagem que deseja encriptar (ou pressione Enter para usar uma aleatória): ")
        if mensagem == "":
            mensagem = get_random_bytes(32)  # Mensagem aleatória de 32 bytes
        else:
            mensagem = mensagem.encode('utf-8')  # Converter para bytes
        sk, ciphertext = encriptar_parametro(parametro_escolhido, mensagem)
        # Converter de lista para bytes, garantindo que os valores estão no intervalo correto
        sk_bytes = lista_para_bytes(sk)
        ciphertext_bytes = lista_para_bytes(ciphertext)
        # Exibir em formato hexadecimal
        print("Ciphertext gerado: ", binascii.hexlify(ciphertext_bytes).decode())
        print("Chave secreta (sk): ", binascii.hexlify(sk_bytes).decode())

    elif acao == 'descriptografar':
        parametro_escolhido = escolher_parametro()
        # Supondo que o usuário tenha o sk e o ciphertext de uma criptografia anterior
        # O usuário deve fornecer a chave secreta e o ciphertext em formato hexadecimal
        sk_input = input("Insira a chave secreta (sk) em hexadecimal: ")
        ciphertext_input = input("Insira o ciphertext em hexadecimal: ")
        # Convertendo de hexadecimal para bytes
        sk = binascii.unhexlify(sk_input)
        ciphertext = binascii.unhexlify(ciphertext_input)
        shared_secret_dec = descriptografar_parametro(parametro_escolhido, sk, ciphertext)
        print("Decapsulação completa. Chave compartilhada:", shared_secret_dec)

# Função de descriptografia para Kyber-512
def kem_decaps512(private_key, ciphertext):
    params_k = 2
    sk = private_key[0: KYBER_INDCPA_SECRETKEY_BYTES_K512]
    pk = private_key[KYBER_INDCPA_SECRETKEY_BYTES_K512:KYBER_INDCPA_SECRETKEY_BYTES_K512 + KYBER_INDCPA_PUBLICKEYBYTES_K512]
    z = private_key[KYBER_512SK_BYTES - KYBER_SYM_BYTES:]
    h = private_key[KYBER_512SK_BYTES - 2 * KYBER_SYM_BYTES:KYBER_512SK_BYTES - KYBER_SYM_BYTES]

    # Convertendo 'h' (bytes) para uma lista de inteiros se necessário
    if isinstance(h, bytes):
        h = list(h)

    # Processa o ciphertext e descriptografa
    m_ = decrypt(ciphertext, sk, params_k)

    # Verifica se m_ é bytes e converte para lista se necessário
    if isinstance(m_, bytes):
        m_ = list(m_)

    # Agora ambos m_ e h são listas, podemos concatená-los
    md512 = SHA3_512.new()
    md512.update(bytearray([x & 0xFF for x in (m_ + h)]))  # Concatena m_ e h

    K_r_ = md512.digest()
    K_r_ = [cast_to_byte(x) for x in K_r_]
    r_ = K_r_[-KYBER_SYM_BYTES:]

    cmp = encrypt(m_, pk, r_, params_k)
    md = SHA3_256.new()
    md.update(bytearray([x & 0xFF for x in ciphertext]))
    Hc = md.digest()
    Hc = [cast_to_byte(x) for x in Hc]

    k = K_r_[:KYBER_SYM_BYTES]

    if cmp == ciphertext:
        temp_buf = k + Hc
    else:
        temp_buf = list(z) + Hc  # Convertendo z para lista, caso não seja

    xof = SHAKE256.new()
    xof.update(bytearray([x & 0xFF for x in temp_buf]))
    sharedSecretFixedLength = xof.read(KYBER_SS_BYTES)

    # Retorna a chave secreta compartilhada
    return [cast_to_byte(x) for x in sharedSecretFixedLength]

if __name__ == "__main__":
    main()


Escolha uma ação:
1 - Criptografar
2 - Descriptografar


Digite o número da sua escolha:  2


Escolha um dos parâmetros Kyber:
1 - Kyber-512
2 - Kyber-512-90s
3 - Kyber-768
4 - Kyber-768-90s
5 - Kyber-1024
6 - Kyber-1024-90s


Digite o número da sua escolha:  1
Insira a chave secreta (sk) em hexadecimal:  905066c9f86a663139bd07b8df40035718412958ab570718cf132e55b42875c54c61fc888aaca10f157223c8983f68485fb357b0cc1781b846282470ef928eb212c7fdf4455aebcf22d989119a7224080c6af5af28b132ba7c7dd781b1ce703fd464a43533ce31973aad5aba7d26c9cb923553d389ee217fb9e531c3e835a31a65e6e0731b471ed68384c787b8b8b883b3c5284043495d2ccf74017572d196a7634de4079f9734cbe3aa3a732325a2ab3e0844177bd8052168b2572b1cef7a3c9f714eec2348b0a762836205da9cc191ca2b3061795ee25f8c9a1c1197071028057be05a9b305fe8a448dae056ed4c357624608a0520663292cf6901925c498034a3711418287431ba02285b9277b7d321213115d06a3e4b1a1c013a84ed499bdf21ac62d7857792c7d737a71a0526ca11a16c231555963cf106c84acc7de7570df734670ea00cc4cc842cc76c9df3c1914a2e28b019831075477b9ab0996d455411379c7e3982b620251359d498a421035a9a636427a83a8669c7009753287a148306f4186bc12a5bfe885ba9d2cab920768ad87d50321372a8673e4b5b9838cfe8a214e233b515d05f34fa806d358db7f63799276858e271f00ab623d59e8562b32084ac207955f5122503

Chave compartilhada decapsulada: [229, 38, 158, 22, 184, 59, 96, 63, 160, 108, 255, 186, 175, 193, 105, 199, 131, 126, 210, 120, 133, 255, 244, 228, 124, 249, 205, 190, 114, 190, 209, 254]
Decapsulação completa. Chave compartilhada: [229, 38, 158, 22, 184, 59, 96, 63, 160, 108, 255, 186, 175, 193, 105, 199, 131, 126, 210, 120, 133, 255, 244, 228, 124, 249, 205, 190, 114, 190, 209, 254]


In [ ]:
from Crypto.Cipher import AES, ChaCha20_Poly1305
from Crypto.Random import get_random_bytes
from Crypto.Protocol.KDF import scrypt
from ccakem import (
    kem_keygen512, kem_keygen768, kem_keygen1024,
    kem_encaps512, kem_encaps768, kem_encaps1024,
    kem_decaps512, kem_decaps768, kem_decaps1024
)
from cpake import decrypt, encrypt  # Importando as funções necessárias para encriptar e descriptografar
from params import (
    KYBER_512SK_BYTES, KYBER_768SK_BYTES, KYBER_1024SK_BYTES, KYBER_SYM_BYTES,
    KYBER_INDCPA_SECRETKEY_BYTES_K512, KYBER_INDCPA_PUBLICKEYBYTES_K512,
    KYBER_INDCPA_SECRETKEY_BYTES_K768, KYBER_INDCPA_PUBLICKEYBYTES_K768,
    KYBER_INDCPA_SECRETKEY_BYTES_K1024, KYBER_INDCPA_PUBLICKEYBYTES_K1024,
    KYBER_SS_BYTES  # Agora incluído o KYBER_SS_BYTES do arquivo params.py
)
import binascii

# Função para converter um valor em byte
def cast_to_byte(x):
    """Converte um valor inteiro para byte (0 a 255)."""
    return x & 0xFF

# Função para converter uma lista de inteiros em bytes
def lista_para_bytes(lista):
    return bytes([x & 0xFF for x in lista])

# Função para exibir menu de escolha de ação (Criptografar ou Descriptografar)
def escolher_acao():
    print("Escolha uma ação:")
    print("1 - Criptografar")
    print("2 - Descriptografar")

    opcao = input("Digite o número da sua escolha: ")

    if opcao == '1':
        return 'criptografar'
    elif opcao == '2':
        return 'descriptografar'
    else:
        print("Opção inválida.")
        return escolher_acao()

# Função para exibir menu de escolha de parâmetros Kyber
def escolher_parametro():
    print("Escolha um dos parâmetros Kyber:")
    print("1 - Kyber-512")
    print("2 - Kyber-512-90s")
    print("3 - Kyber-768")
    print("4 - Kyber-768-90s")
    print("5 - Kyber-1024")
    print("6 - Kyber-1024-90s")

    opcao = input("Digite o número da sua escolha: ")

    if opcao == '1':
        return 512
    elif opcao == '2':
        return '512-90s'
    elif opcao == '3':
        return 768
    elif opcao == '4':
        return '768-90s'
    elif opcao == '5':
        return 1024
    elif opcao == '6':
        return '1024-90s'
    else:
        print("Opção inválida.")
        return escolher_parametro()

# Função para o usuário escolher o algoritmo simétrico
def escolher_algoritmo_simetrico():
    print("Escolha um algoritmo simétrico para criptografar a mensagem:")
    print("1 - AES-GCM")
    print("2 - ChaCha20-Poly1305")

    opcao = input("Digite o número da sua escolha: ")

    if opcao == '1':
        return 'AES-GCM'
    elif opcao == '2':
        return 'ChaCha20-Poly1305'
    else:
        print("Opção inválida.")
        return escolher_algoritmo_simetrico()

# Função para criptografar a mensagem usando AES-GCM
def aes_encrypt(shared_secret, mensagem):
    # Converter a lista de inteiros em bytes
    shared_secret_bytes = lista_para_bytes(shared_secret)

    # Derivar uma chave AES da chave compartilhada usando Scrypt (ou outro KDF)
    aes_key = scrypt(shared_secret_bytes, b'salt', 16, N=2**14, r=8, p=1)

    # Criar uma instância do AES-GCM para criptografia
    cipher = AES.new(aes_key, AES.MODE_GCM)
    nonce = cipher.nonce
    ciphertext, tag = cipher.encrypt_and_digest(mensagem)

    return nonce, ciphertext, tag

# Função para descriptografar a mensagem usando AES-GCM
def aes_decrypt(shared_secret, nonce, ciphertext, tag):
    # Converter a lista de inteiros em bytes
    shared_secret_bytes = lista_para_bytes(shared_secret)

    # Derivar a mesma chave AES da chave compartilhada
    aes_key = scrypt(shared_secret_bytes, b'salt', 16, N=2**14, r=8, p=1)

    # Criar uma instância do AES-GCM para descriptografia
    cipher = AES.new(aes_key, AES.MODE_GCM, nonce=nonce)
    mensagem = cipher.decrypt_and_verify(ciphertext, tag)

    return mensagem

# Função para criptografar usando ChaCha20-Poly1305
def chacha20_encrypt(shared_secret, mensagem):
    # Converter a lista de inteiros em bytes
    shared_secret_bytes = lista_para_bytes(shared_secret)

    # Derivar uma chave ChaCha20 da chave compartilhada usando Scrypt
    chacha_key = scrypt(shared_secret_bytes, b'salt', 32, N=2**14, r=8, p=1)

    # Criação da instância ChaCha20-Poly1305
    cipher = ChaCha20_Poly1305.new(key=chacha_key)
    nonce = cipher.nonce
    ciphertext, tag = cipher.encrypt_and_digest(mensagem)

    return nonce, ciphertext, tag

# Função para descriptografar usando ChaCha20-Poly1305
def chacha20_decrypt(shared_secret, nonce, ciphertext, tag):
    # Converter a lista de inteiros em bytes
    shared_secret_bytes = lista_para_bytes(shared_secret)

    # Derivar a mesma chave ChaCha20 da chave compartilhada
    chacha_key = scrypt(shared_secret_bytes, b'salt', 32, N=2**14, r=8, p=1)

    # Instância ChaCha20-Poly1305 para descriptografia
    cipher = ChaCha20_Poly1305.new(key=chacha_key, nonce=nonce)
    mensagem = cipher.decrypt_and_verify(ciphertext, tag)

    return mensagem

# Função para gerar chave e realizar encapsulamento com o parâmetro escolhido
def encriptar_parametro_com_mensagem(parametro, algoritmo, mensagem):
    if parametro == 512:
        sk, pk = kem_keygen512()
        print("Chave gerada para Kyber-512")
        shared_secret, ciphertext = kem_encaps512(pk)
    elif parametro == 768:
        sk, pk = kem_keygen768()
        print("Chave gerada para Kyber-768")
        shared_secret, ciphertext = kem_encaps768(pk)
    elif parametro == 1024:
        sk, pk = kem_keygen1024()
        print("Chave gerada para Kyber-1024")
        shared_secret, ciphertext = kem_encaps1024(pk)
    else:
        print("Somente os modos normais de Kyber foram implementados no momento.")
        return

    # Exibir chave compartilhada e ciphertext em hexadecimal
    shared_secret_hex = binascii.hexlify(lista_para_bytes(shared_secret)).decode()
    ciphertext_hex = binascii.hexlify(lista_para_bytes(ciphertext)).decode()

    print("Chave compartilhada (hex):", shared_secret_hex)
    print("Ciphertext (hex):", ciphertext_hex)

    # Agora criptografamos a mensagem com o algoritmo simétrico escolhido
    if algoritmo == 'AES-GCM':
        nonce, encrypted_message, tag = aes_encrypt(shared_secret, mensagem)
    elif algoritmo == 'ChaCha20-Poly1305':
        nonce, encrypted_message, tag = chacha20_encrypt(shared_secret, mensagem)

    return sk, ciphertext, nonce, encrypted_message, tag

# Função para descriptografar a mensagem criptografada com o algoritmo simétrico escolhido
def descriptografar_com_mensagem(parametro, algoritmo, sk, ciphertext, nonce, encrypted_message, tag):
    if parametro == 512:
        shared_secret_dec = kem_decaps512(sk, ciphertext)
    elif parametro == 768:
        shared_secret_dec = kem_decaps768(sk, ciphertext)
    elif parametro == 1024:
        shared_secret_dec = kem_decaps1024(sk, ciphertext)
    else:
        print("Somente os modos normais de Kyber foram implementados no momento.")
        return

    # Exibir chave compartilhada decapsulada em hexadecimal
    shared_secret_dec_hex = binascii.hexlify(lista_para_bytes(shared_secret_dec)).decode()
    print("Chave compartilhada decapsulada (hex):", shared_secret_dec_hex)

    # Agora descriptografamos a mensagem usando o algoritmo simétrico escolhido
    if algoritmo == 'AES-GCM':
        decrypted_message = aes_decrypt(shared_secret_dec, nonce, encrypted_message, tag)
    elif algoritmo == 'ChaCha20-Poly1305':
        decrypted_message = chacha20_decrypt(shared_secret_dec, nonce, encrypted_message, tag)

    return decrypted_message

# Função principal para escolher a ação e o modo Kyber e realizar o processo
def main():
    acao = escolher_acao()

    if acao == 'criptografar':
        parametro_escolhido = escolher_parametro()
        algoritmo_escolhido = escolher_algoritmo_simetrico()
        mensagem = input("Digite a mensagem que deseja encriptar: ").encode('utf-8')
        sk, ciphertext, nonce, encrypted_message, tag = encriptar_parametro_com_mensagem(parametro_escolhido, algoritmo_escolhido, mensagem)
        print(f"Mensagem criptografada: {binascii.hexlify(encrypted_message).decode()}")

    elif acao == 'descriptografar':
        parametro_escolhido = escolher_parametro()
        algoritmo_escolhido = escolher_algoritmo_simetrico()
        sk_input = input("Insira a chave secreta (sk) em hexadecimal: ")
        ciphertext_input = input("Insira o ciphertext em hexadecimal: ")
        nonce_input = input("Insira o nonce em hexadecimal: ")
        encrypted_message_input = input("Insira a mensagem criptografada em hexadecimal: ")
        tag_input = input("Insira o tag de autenticação em hexadecimal: ")

        # Convertendo os dados de hexadecimal para bytes
        sk = binascii.unhexlify(sk_input)
        ciphertext = binascii.unhexlify(ciphertext_input)
        nonce = binascii.unhexlify(nonce_input)
        encrypted_message = binascii.unhexlify(encrypted_message_input)
        tag = binascii.unhexlify(tag_input)

        decrypted_message = descriptografar_com_mensagem(parametro_escolhido, algoritmo_escolhido, sk, ciphertext, nonce, encrypted_message, tag)
        print(f"Mensagem original descriptografada: {decrypted_message.decode()}")

if __name__ == "__main__":
    main()

Escolha uma ação:
1 - Criptografar
2 - Descriptografar


Digite o número da sua escolha:  2


Escolha um dos parâmetros Kyber:
1 - Kyber-512
2 - Kyber-512-90s
3 - Kyber-768
4 - Kyber-768-90s
5 - Kyber-1024
6 - Kyber-1024-90s


Digite o número da sua escolha:  1


Escolha um algoritmo simétrico para criptografar a mensagem:
1 - AES-GCM
2 - ChaCha20-Poly1305


Digite o número da sua escolha:  1
Insira a chave secreta (sk) em hexadecimal:  ad92b900dffa929d30655f81e1680e8f4e8e7120d31bcf9c06fea2def06b031400d9beaae444ea97a4a918c9cfc13391809ea5875cfd989df04ae44f3e4414d9b5eaee68bddffac558494d4101041eac0f1193a564b9d3f31e9fee7a77c7d33da1b7d2effc7177c8db088a5223cb44e179931004610b8a92514494c3a6ef4aee100911c47a1585d39efad589b5d64ab4f8b98313b2c263a2c59088ff19007f9be4333a791fe2ee4f8e3f17f8459c4fb922a5b1afe280233e0b40d20b73ddbbcdbf7726286d170a6a071315b6e638577bee79fdeae6ae5ab2c4f87a7b0605387de61ecd9196ab010776b626d9b732bc2d30d777ebe80c7af12aef3992c82931c0f0029ce43143703467e64b4327ad9bf1d9694a8b057bbdfd0110b7f66e2d215033698c4fd2ac23fc088f1ef7d79a0fcc3b132ec93c962fb261e8475ab0d649d7740b9ea713ad8b676000bf48dc49e943ab3df98ab0b9cf47b8ecc8eeeaf6190b1fce7a4dda6f8a4654094c5f41c0c50a609102bb5fe1c91753c05b0202e7a7cc5ec6397678296fff1e6646c9a9b5f0bebc97257d0de2bebd0ac21c15187680d490da0c39946ce65eecde6d2aba379187755d9ab1687af7f50fe4974020db2b66a67b96d2e2f2078442131101

KeyboardInterrupt: Interrupted by user

In [ ]:
from Crypto.Cipher import AES, ChaCha20_Poly1305
from Crypto.Random import get_random_bytes
from Crypto.Protocol.KDF import scrypt
from ccakem import (
    kem_keygen512, kem_keygen768, kem_keygen1024,
    kem_encaps512, kem_encaps768, kem_encaps1024,
    kem_decaps512, kem_decaps768, kem_decaps1024
)
from cpake import decrypt, encrypt  # Importando as funções necessárias para encriptar e descriptografar
from params import (
    KYBER_512SK_BYTES, KYBER_768SK_BYTES, KYBER_1024SK_BYTES, KYBER_SYM_BYTES,
    KYBER_INDCPA_SECRETKEY_BYTES_K512, KYBER_INDCPA_PUBLICKEYBYTES_K512,
    KYBER_INDCPA_SECRETKEY_BYTES_K768, KYBER_INDCPA_PUBLICKEYBYTES_K768,
    KYBER_INDCPA_SECRETKEY_BYTES_K1024, KYBER_INDCPA_PUBLICKEYBYTES_K1024,
    KYBER_SS_BYTES  # Agora incluído o KYBER_SS_BYTES do arquivo params.py
)
import binascii

# Função para converter um valor em byte
def cast_to_byte(x):
    """Converte um valor inteiro para byte (0 a 255)."""
    return x & 0xFF

# Função para converter uma lista de inteiros em bytes
def lista_para_bytes(lista):
    return bytes([x & 0xFF for x in lista])

# Função para exibir menu de escolha de ação (Criptografar ou Descriptografar)
def escolher_acao():
    print("Escolha uma ação:")
    print("1 - Criptografar")
    print("2 - Descriptografar")

    opcao = input("Digite o número da sua escolha: ")

    if opcao == '1':
        return 'criptografar'
    elif opcao == '2':
        return 'descriptografar'
    else:
        print("Opção inválida.")
        return escolher_acao()

# Função para exibir menu de escolha de parâmetros Kyber
def escolher_parametro():
    print("Escolha um dos parâmetros Kyber:")
    print("1 - Kyber-512")
    print("2 - Kyber-512-90s")
    print("3 - Kyber-768")
    print("4 - Kyber-768-90s")
    print("5 - Kyber-1024")
    print("6 - Kyber-1024-90s")

    opcao = input("Digite o número da sua escolha: ")

    if opcao == '1':
        return 512
    elif opcao == '2':
        return '512-90s'
    elif opcao == '3':
        return 768
    elif opcao == '4':
        return '768-90s'
    elif opcao == '5':
        return 1024
    elif opcao == '6':
        return '1024-90s'
    else:
        print("Opção inválida.")
        return escolher_parametro()

# Função para o usuário escolher o algoritmo simétrico
def escolher_algoritmo_simetrico():
    print("Escolha um algoritmo simétrico para criptografar a mensagem:")
    print("1 - AES-GCM")
    print("2 - ChaCha20-Poly1305")

    opcao = input("Digite o número da sua escolha: ")

    if opcao == '1':
        return 'AES-GCM'
    elif opcao == '2':
        return 'ChaCha20-Poly1305'
    else:
        print("Opção inválida.")
        return escolher_algoritmo_simetrico()

# Função para criptografar a mensagem usando AES-GCM
def aes_encrypt(shared_secret, mensagem):
    # Converter a lista de inteiros em bytes
    shared_secret_bytes = lista_para_bytes(shared_secret)

    # Derivar uma chave AES da chave compartilhada usando Scrypt (ou outro KDF)
    aes_key = scrypt(shared_secret_bytes, b'salt', 16, N=2**14, r=8, p=1)

    # Criar uma instância do AES-GCM para criptografia
    cipher = AES.new(aes_key, AES.MODE_GCM)
    nonce = cipher.nonce
    ciphertext, tag = cipher.encrypt_and_digest(mensagem)

    return nonce, ciphertext, tag

# Função para descriptografar a mensagem usando AES-GCM
def aes_decrypt(shared_secret, nonce, ciphertext, tag):
    # Converter a lista de inteiros em bytes
    shared_secret_bytes = lista_para_bytes(shared_secret)

    # Derivar a mesma chave AES da chave compartilhada
    aes_key = scrypt(shared_secret_bytes, b'salt', 16, N=2**14, r=8, p=1)

    # Criar uma instância do AES-GCM para descriptografia
    cipher = AES.new(aes_key, AES.MODE_GCM, nonce=nonce)
    mensagem = cipher.decrypt_and_verify(ciphertext, tag)

    return mensagem

# Função para criptografar usando ChaCha20-Poly1305
def chacha20_encrypt(shared_secret, mensagem):
    # Converter a lista de inteiros em bytes
    shared_secret_bytes = lista_para_bytes(shared_secret)

    # Derivar uma chave ChaCha20 da chave compartilhada usando Scrypt
    chacha_key = scrypt(shared_secret_bytes, b'salt', 32, N=2**14, r=8, p=1)

    # Criação da instância ChaCha20-Poly1305
    cipher = ChaCha20_Poly1305.new(key=chacha_key)
    nonce = cipher.nonce
    ciphertext, tag = cipher.encrypt_and_digest(mensagem)

    return nonce, ciphertext, tag

# Função para descriptografar usando ChaCha20-Poly1305
def chacha20_decrypt(shared_secret, nonce, ciphertext, tag):
    # Converter a lista de inteiros em bytes
    shared_secret_bytes = lista_para_bytes(shared_secret)

    # Derivar a mesma chave ChaCha20 da chave compartilhada
    chacha_key = scrypt(shared_secret_bytes, b'salt', 32, N=2**14, r=8, p=1)

    # Instância ChaCha20-Poly1305 para descriptografia
    cipher = ChaCha20_Poly1305.new(key=chacha_key, nonce=nonce)
    mensagem = cipher.decrypt_and_verify(ciphertext, tag)

    return mensagem

# Função para gerar chave e realizar encapsulamento com o parâmetro escolhido
def encriptar_parametro_com_mensagem(parametro, algoritmo, mensagem):
    if parametro == 512:
        sk, pk = kem_keygen512()
        print("Chave gerada para Kyber-512")
        shared_secret, ciphertext = kem_encaps512(pk)
    elif parametro == 768:
        sk, pk = kem_keygen768()
        print("Chave gerada para Kyber-768")
        shared_secret, ciphertext = kem_encaps768(pk)
    elif parametro == 1024:
        sk, pk = kem_keygen1024()
        print("Chave gerada para Kyber-1024")
        shared_secret, ciphertext = kem_encaps1024(pk)
    else:
        print("Somente os modos normais de Kyber foram implementados no momento.")
        return

    # Exibir chave compartilhada e ciphertext em hexadecimal
    shared_secret_hex = binascii.hexlify(lista_para_bytes(shared_secret)).decode()
    ciphertext_hex = binascii.hexlify(lista_para_bytes(ciphertext)).decode()
    sk_hex = binascii.hexlify(lista_para_bytes(sk)).decode()

    print("Chave secreta (sk) em hexadecimal:", sk_hex)
    print("Ciphertext em hexadecimal:", ciphertext_hex)
    print("Chave compartilhada (hex):", shared_secret_hex)

    # Agora criptografamos a mensagem com o algoritmo simétrico escolhido
    if algoritmo == 'AES-GCM':
        nonce, encrypted_message, tag = aes_encrypt(shared_secret, mensagem)
    elif algoritmo == 'ChaCha20-Poly1305':
        nonce, encrypted_message, tag = chacha20_encrypt(shared_secret, mensagem)

    # Exibir a mensagem criptografada e o tag de autenticação em hexadecimal
    encrypted_message_hex = binascii.hexlify(encrypted_message).decode()
    tag_hex = binascii.hexlify(tag).decode()
    nonce_hex = binascii.hexlify(nonce).decode()

    print("Mensagem criptografada em hexadecimal:", encrypted_message_hex)
    print("Tag de autenticação em hexadecimal:", tag_hex)
    print("Nonce em hexadecimal:", nonce_hex)

    return sk, ciphertext, nonce, encrypted_message, tag

# Função para descriptografar a mensagem criptografada com o algoritmo simétrico escolhido
def descriptografar_com_mensagem(parametro, algoritmo, sk, ciphertext, nonce, encrypted_message, tag):
    if parametro == 512:
        shared_secret_dec = kem_decaps512(sk, ciphertext)
    elif parametro == 768:
        shared_secret_dec = kem_decaps768(sk, ciphertext)
    elif parametro == 1024:
        shared_secret_dec = kem_decaps1024(sk, ciphertext)
    else:
        print("Somente os modos normais de Kyber foram implementados no momento.")
        return

    # Exibir chave compartilhada decapsulada em hexadecimal
    shared_secret_dec_hex = binascii.hexlify(lista_para_bytes(shared_secret_dec)).decode()
    print("Chave compartilhada decapsulada (hex):", shared_secret_dec_hex)

    # Agora descriptografamos a mensagem usando o algoritmo simétrico escolhido
    if algoritmo == 'AES-GCM':
        decrypted_message = aes_decrypt(shared_secret_dec, nonce, encrypted_message, tag)
    elif algoritmo == 'ChaCha20-Poly1305':
        decrypted_message = chacha20_decrypt(shared_secret_dec, nonce, encrypted_message, tag)

    return decrypted_message

# Função principal para escolher a ação e o modo Kyber e realizar o processo
def main():
    acao = escolher_acao()

    if acao == 'criptografar':
        parametro_escolhido = escolher_parametro()
        algoritmo_escolhido = escolher_algoritmo_simetrico()
        mensagem = input("Digite a mensagem que deseja encriptar: ").encode('utf-8')
        sk, ciphertext, nonce, encrypted_message, tag = encriptar_parametro_com_mensagem(parametro_escolhido, algoritmo_escolhido, mensagem)

    elif acao == 'descriptografar':
        parametro_escolhido = escolher_parametro()
        algoritmo_escolhido = escolher_algoritmo_simetrico()
        sk_input = input("Insira a chave secreta (sk) em hexadecimal: ")
        ciphertext_input = input("Insira o ciphertext em hexadecimal: ")
        nonce_input = input("Insira o nonce em hexadecimal: ")
        encrypted_message_input = input("Insira a mensagem criptografada em hexadecimal: ")
        tag_input = input("Insira o tag de autenticação em hexadecimal: ")

        # Convertendo os dados de hexadecimal para bytes
        sk = binascii.unhexlify(sk_input)
        ciphertext = binascii.unhexlify(ciphertext_input)
        nonce = binascii.unhexlify(nonce_input)
        encrypted_message = binascii.unhexlify(encrypted_message_input)
        tag = binascii.unhexlify(tag_input)

        decrypted_message = descriptografar_com_mensagem(parametro_escolhido, algoritmo_escolhido, sk, ciphertext, nonce, encrypted_message, tag)
        print(f"Mensagem original descriptografada: {decrypted_message.decode()}")

if __name__ == "__main__":
    main()


Escolha uma ação:
1 - Criptografar
2 - Descriptografar


Digite o número da sua escolha:  2


Escolha um dos parâmetros Kyber:
1 - Kyber-512
2 - Kyber-512-90s
3 - Kyber-768
4 - Kyber-768-90s
5 - Kyber-1024
6 - Kyber-1024-90s


Digite o número da sua escolha:  1


Escolha um algoritmo simétrico para criptografar a mensagem:
1 - AES-GCM
2 - ChaCha20-Poly1305


Digite o número da sua escolha:  1
Insira a chave secreta (sk) em hexadecimal:  5dccad1e84c5585256d3b6426fcb87b9667d9ac19cbf75860fe93c397289ac627e178270405203c2b55b34178f791429c4c4674a8957ffa971f3b374dd9aa34b11aa4336b303280ba481692338cff4360a580b770af23a2987736589a7137c36484219518391b6045efd9098bf571b54ecbca917352cc7ca98d4c70951a769b94cf39a95597a955ccccb92034321f6709c815719eb75f58733a98c16703615629c8ba3095447942dcb7775a2d14fd7a4424e80a9a490c89c04829241b303e4acd01359cea5cd0d3a7281766782037457b2463f096617da8d0f161c920518f2068fab3b273fac799bd1773db2669700caad4b10e390c26fd7a5363a97bbd9b04273284d3759f6065ed7857bfcf293f1d3c7a4d99a60b16196c7686ed680135606d3e35f8ab79c8a956786b23157e956ba0906f3d4c3f1950981889fcd48a74e972e85db57e2351c2a80730106c425e65ad0e08c0616ce89634164c88af85c611b8232af97a85c234336d255bfb2080557bbefb7690ee5a2cee68fb48600f7f9150bb14e790565ac2337e8504dc9d983256aab032710139b1a131c5723278eceeab40a0453eb542bedc375ab61382fb81fe81a77d947637c817357e9413dbb342c297b9d316dc94148edb3a8ab

TypeError: can only concatenate list (not "bytes") to list

In [ ]:
from Crypto.Cipher import AES, ChaCha20_Poly1305
from Crypto.Random import get_random_bytes
from Crypto.Protocol.KDF import scrypt
from ccakem import (
    kem_keygen512, kem_keygen768, kem_keygen1024,
    kem_encaps512, kem_encaps768, kem_encaps1024,
    kem_decaps512, kem_decaps768, kem_decaps1024
)
from cpake import decrypt, encrypt  # Importando as funções necessárias para encriptar e descriptografar
from params import (
    KYBER_512SK_BYTES, KYBER_768SK_BYTES, KYBER_1024SK_BYTES, KYBER_SYM_BYTES,
    KYBER_INDCPA_SECRETKEY_BYTES_K512, KYBER_INDCPA_PUBLICKEYBYTES_K512,
    KYBER_INDCPA_SECRETKEY_BYTES_K768, KYBER_INDCPA_PUBLICKEYBYTES_K768,
    KYBER_INDCPA_SECRETKEY_BYTES_K1024, KYBER_INDCPA_PUBLICKEYBYTES_K1024,
    KYBER_SS_BYTES  # Agora incluído o KYBER_SS_BYTES do arquivo params.py
)
import binascii

# Função para converter um valor em byte
def cast_to_byte(x):
    """Converte um valor inteiro para byte (0 a 255)."""
    return x & 0xFF

# Função para converter uma lista de inteiros em bytes
def lista_para_bytes(lista):
    return bytes([x & 0xFF for x in lista])

# Função para exibir menu de escolha de ação (Criptografar ou Descriptografar)
def escolher_acao():
    print("Escolha uma ação:")
    print("1 - Criptografar")
    print("2 - Descriptografar")

    opcao = input("Digite o número da sua escolha: ")

    if opcao == '1':
        return 'criptografar'
    elif opcao == '2':
        return 'descriptografar'
    else:
        print("Opção inválida.")
        return escolher_acao()

# Função para exibir menu de escolha de parâmetros Kyber
def escolher_parametro():
    print("Escolha um dos parâmetros Kyber:")
    print("1 - Kyber-512")
    print("2 - Kyber-512-90s")
    print("3 - Kyber-768")
    print("4 - Kyber-768-90s")
    print("5 - Kyber-1024")
    print("6 - Kyber-1024-90s")

    opcao = input("Digite o número da sua escolha: ")

    if opcao == '1':
        return 512
    elif opcao == '2':
        return '512-90s'
    elif opcao == '3':
        return 768
    elif opcao == '4':
        return '768-90s'
    elif opcao == '5':
        return 1024
    elif opcao == '6':
        return '1024-90s'
    else:
        print("Opção inválida.")
        return escolher_parametro()

# Função para o usuário escolher o algoritmo simétrico
def escolher_algoritmo_simetrico():
    print("Escolha um algoritmo simétrico para criptografar a mensagem:")
    print("1 - AES-GCM")
    print("2 - ChaCha20-Poly1305")

    opcao = input("Digite o número da sua escolha: ")

    if opcao == '1':
        return 'AES-GCM'
    elif opcao == '2':
        return 'ChaCha20-Poly1305'
    else:
        print("Opção inválida.")
        return escolher_algoritmo_simetrico()

# Usando um salt fixo para garantir que a derivação da chave seja a mesma
salt = b'fixed_salt_value'

# Função para criptografar a mensagem usando AES-GCM
def aes_encrypt(shared_secret, mensagem):
    # Converter a lista de inteiros em bytes
    shared_secret_bytes = lista_para_bytes(shared_secret)

    # Derivar uma chave AES da chave compartilhada usando Scrypt (com salt fixo)
    aes_key = scrypt(shared_secret_bytes, salt, 16, N=2**14, r=8, p=1)

    # Criar uma instância do AES-GCM para criptografia
    cipher = AES.new(aes_key, AES.MODE_GCM)
    nonce = cipher.nonce
    ciphertext, tag = cipher.encrypt_and_digest(mensagem)

    # Exibir valores para depuração
    print("=== Criptografia ===")
    print("Chave compartilhada (hex):", binascii.hexlify(shared_secret_bytes).decode())
    print("Nonce (hex):", binascii.hexlify(nonce).decode())
    print("Ciphertext (hex):", binascii.hexlify(ciphertext).decode())
    print("Tag (hex):", binascii.hexlify(tag).decode())

    return nonce, ciphertext, tag

# Função para descriptografar a mensagem usando AES-GCM
def aes_decrypt(shared_secret, nonce, ciphertext, tag):
    # Converter a lista de inteiros em bytes
    shared_secret_bytes = lista_para_bytes(shared_secret)

    # Derivar a mesma chave AES da chave compartilhada usando Scrypt (com salt fixo)
    aes_key = scrypt(shared_secret_bytes, salt, 16, N=2**14, r=8, p=1)

    # Exibir valores para depuração
    print("=== Descriptografia ===")
    print("Chave compartilhada (hex):", binascii.hexlify(shared_secret_bytes).decode())
    print("Nonce (hex):", binascii.hexlify(nonce).decode())
    print("Ciphertext (hex):", binascii.hexlify(ciphertext).decode())
    print("Tag (hex):", binascii.hexlify(tag).decode())

    # Criar uma instância do AES-GCM para descriptografia
    cipher = AES.new(aes_key, AES.MODE_GCM, nonce=nonce)

    try:
        mensagem = cipher.decrypt_and_verify(ciphertext, tag)
        print("Mensagem descriptografada:", mensagem.decode())
        return mensagem
    except ValueError as e:
        print("Erro na descriptografia (MAC check failed):", str(e))
        raise e

# Função para criptografar usando ChaCha20-Poly1305
def chacha20_encrypt(shared_secret, mensagem):
    # Converter a lista de inteiros em bytes
    shared_secret_bytes = lista_para_bytes(shared_secret)

    # Derivar uma chave ChaCha20 da chave compartilhada usando Scrypt
    chacha_key = scrypt(shared_secret_bytes, b'salt', 32, N=2**14, r=8, p=1)

    # Criação da instância ChaCha20-Poly1305
    cipher = ChaCha20_Poly1305.new(key=chacha_key)
    nonce = cipher.nonce
    ciphertext, tag = cipher.encrypt_and_digest(mensagem)

    return nonce, ciphertext, tag

# Função para descriptografar usando ChaCha20-Poly1305
def chacha20_decrypt(shared_secret, nonce, ciphertext, tag):
    # Converter a lista de inteiros em bytes
    shared_secret_bytes = lista_para_bytes(shared_secret)

    # Derivar a mesma chave ChaCha20 da chave compartilhada
    chacha_key = scrypt(shared_secret_bytes, b'salt', 32, N=2**14, r=8, p=1)

    # Instância ChaCha20-Poly1305 para descriptografia
    cipher = ChaCha20_Poly1305.new(key=chacha_key, nonce=nonce)
    mensagem = cipher.decrypt_and_verify(ciphertext, tag)

    return mensagem

# Função para gerar chave e realizar encapsulamento com o parâmetro escolhido
def encriptar_parametro_com_mensagem(parametro, algoritmo, mensagem):
    if parametro == 512:
        sk, pk = kem_keygen512()
        print("Chave gerada para Kyber-512")
        shared_secret, ciphertext = kem_encaps512(pk)
    elif parametro == 768:
        sk, pk = kem_keygen768()
        print("Chave gerada para Kyber-768")
        shared_secret, ciphertext = kem_encaps768(pk)
    elif parametro == 1024:
        sk, pk = kem_keygen1024()
        print("Chave gerada para Kyber-1024")
        shared_secret, ciphertext = kem_encaps1024(pk)
    else:
        print("Somente os modos normais de Kyber foram implementados no momento.")
        return

    # Exibir chave compartilhada e ciphertext em hexadecimal
    shared_secret_hex = binascii.hexlify(lista_para_bytes(shared_secret)).decode()
    ciphertext_hex = binascii.hexlify(lista_para_bytes(ciphertext)).decode()
    sk_hex = binascii.hexlify(lista_para_bytes(sk)).decode()

    print("Chave secreta (sk) em hexadecimal:", sk_hex)
    print("Ciphertext em hexadecimal:", ciphertext_hex)
    print("Chave compartilhada (hex):", shared_secret_hex)

    # Agora criptografamos a mensagem com o algoritmo simétrico escolhido
    if algoritmo == 'AES-GCM':
        nonce, encrypted_message, tag = aes_encrypt(shared_secret, mensagem)
    elif algoritmo == 'ChaCha20-Poly1305':
        nonce, encrypted_message, tag = chacha20_encrypt(shared_secret, mensagem)

    # Exibir a mensagem criptografada e o tag de autenticação em hexadecimal
    encrypted_message_hex = binascii.hexlify(encrypted_message).decode()
    tag_hex = binascii.hexlify(tag).decode()
    nonce_hex = binascii.hexlify(nonce).decode()

    print("Mensagem criptografada em hexadecimal:", encrypted_message_hex)
    print("Tag de autenticação em hexadecimal:", tag_hex)
    print("Nonce em hexadecimal:", nonce_hex)

    return sk, ciphertext, nonce, encrypted_message, tag

# Função para descriptografar a mensagem criptografada com o algoritmo simétrico escolhido
def descriptografar_com_mensagem(parametro, algoritmo, sk, ciphertext, nonce, encrypted_message, tag):
    if parametro == 512:
        shared_secret_dec = kem_decaps512(sk, ciphertext)
    elif parametro == 768:
        shared_secret_dec = kem_decaps768(sk, ciphertext)
    elif parametro == 1024:
        shared_secret_dec = kem_decaps1024(sk, ciphertext)
    else:
        print("Somente os modos normais de Kyber foram implementados no momento.")
        return

    # Exibir chave compartilhada decapsulada em hexadecimal
    shared_secret_dec_hex = binascii.hexlify(lista_para_bytes(shared_secret_dec)).decode()
    print("Chave compartilhada decapsulada (hex):", shared_secret_dec_hex)

    # Agora descriptografamos a mensagem usando o algoritmo simétrico escolhido
    if algoritmo == 'AES-GCM':
        decrypted_message = aes_decrypt(shared_secret_dec, nonce, encrypted_message, tag)
    elif algoritmo == 'ChaCha20-Poly1305':
        decrypted_message = chacha20_decrypt(shared_secret_dec, nonce, encrypted_message, tag)

    return decrypted_message

# Função para decapsular a chave Kyber-512 e corrigida para concatenar listas
def kem_decaps512(private_key, ciphertext):
    params_k = 2
    sk = private_key[0: KYBER_INDCPA_SECRETKEY_BYTES_K512]
    pk = private_key[KYBER_INDCPA_SECRETKEY_BYTES_K512:KYBER_INDCPA_SECRETKEY_BYTES_K512 + KYBER_INDCPA_PUBLICKEYBYTES_K512]
    z = private_key[KYBER_512SK_BYTES - KYBER_SYM_BYTES:]
    h = private_key[KYBER_512SK_BYTES - 2 * KYBER_SYM_BYTES:KYBER_512SK_BYTES - KYBER_SYM_BYTES]

    # Convertendo 'h' para uma lista de inteiros se necessário
    if isinstance(h, bytes):
        h = list(h)

    # Processa o ciphertext e descriptografa
    m_ = decrypt(ciphertext, sk, params_k)

    # Convertendo 'm_' para uma lista de inteiros, caso seja bytes
    if isinstance(m_, bytes):
        m_ = list(m_)

    # Agora concatenamos as listas de inteiros
    md512 = SHA3_512.new()
    md512.update(bytearray([x & 0xFF for x in (m_ + h)]))  # Concatena m_ e h

    K_r_ = md512.digest()
    K_r_ = [cast_to_byte(x) for x in K_r_]
    r_ = K_r_[-KYBER_SYM_BYTES:]

    cmp = encrypt(m_, pk, r_, params_k)
    md = SHA3_256.new()
    md.update(bytearray([x & 0xFF for x in ciphertext]))
    Hc = md.digest()
    Hc = [cast_to_byte(x) for x in Hc]

    k = K_r_[:KYBER_SYM_BYTES]

    if cmp == ciphertext:
        temp_buf = k + Hc
    else:
        temp_buf = list(z) + Hc  # Convertendo z para lista, caso ainda não seja

    xof = SHAKE256.new()
    xof.update(bytearray([x & 0xFF for x in temp_buf]))
    sharedSecretFixedLength = xof.read(KYBER_SS_BYTES)

    # Retorna a chave secreta compartilhada
    return [cast_to_byte(x) for x in sharedSecretFixedLength]

# Função principal para escolher a ação e o modo Kyber e realizar o processo
def main():
    acao = escolher_acao()

    if acao == 'criptografar':
        parametro_escolhido = escolher_parametro()
        algoritmo_escolhido = escolher_algoritmo_simetrico()
        mensagem = input("Digite a mensagem que deseja encriptar: ").encode('utf-8')
        sk, ciphertext, nonce, encrypted_message, tag = encriptar_parametro_com_mensagem(parametro_escolhido, algoritmo_escolhido, mensagem)

    elif acao == 'descriptografar':
        parametro_escolhido = escolher_parametro()
        algoritmo_escolhido = escolher_algoritmo_simetrico()
        sk_input = input("Insira a chave secreta (sk) em hexadecimal: ")
        ciphertext_input = input("Insira o ciphertext em hexadecimal: ")
        nonce_input = input("Insira o nonce em hexadecimal: ")
        encrypted_message_input = input("Insira a mensagem criptografada em hexadecimal: ")
        tag_input = input("Insira o tag de autenticação em hexadecimal: ")

        # Convertendo os dados de hexadecimal para bytes
        sk = binascii.unhexlify(sk_input)
        ciphertext = binascii.unhexlify(ciphertext_input)
        nonce = binascii.unhexlify(nonce_input)
        encrypted_message = binascii.unhexlify(encrypted_message_input)
        tag = binascii.unhexlify(tag_input)

        decrypted_message = descriptografar_com_mensagem(parametro_escolhido, algoritmo_escolhido, sk, ciphertext, nonce, encrypted_message, tag)
        print(f"Mensagem original descriptografada: {decrypted_message.decode()}")

if __name__ == "__main__":
    main()


Escolha uma ação:
1 - Criptografar
2 - Descriptografar


Digite o número da sua escolha:  2


Escolha um dos parâmetros Kyber:
1 - Kyber-512
2 - Kyber-512-90s
3 - Kyber-768
4 - Kyber-768-90s
5 - Kyber-1024
6 - Kyber-1024-90s


Digite o número da sua escolha:  1


Escolha um algoritmo simétrico para criptografar a mensagem:
1 - AES-GCM
2 - ChaCha20-Poly1305


Digite o número da sua escolha:  1
Insira a chave secreta (sk) em hexadecimal:  7745aec593b02c7abd7c669ca7b0a7c2d75f477439f21060b3380f2917326b33c3ffcb1a05e5cf7d56c188333ff80b32f7e7346b43aeafd95d96584a0d2c906be7c13ce07f5e4a8744b1cc6165beb5f5a71e369003ab7c12876762056812b65911e3b6f072cdf9bc8b200b5ac4dc4f7036ccc07a486b117d3e84c3b207138ff642b23239d61785edf438568678e907ba272c6f650bad57f3900f020fd9d0544353cd63e8273eba785a822607292c14945c48027238e0bfe69a3b5c7c832755a4b4c93aab3b2d091b216612a39e01c73a4c57c4d88ac02132542138dfd43f8ffa5b5d3b81eeec9cb4ec1b4f93cf140c79b1129737f61e1bc0736a383f04e81a028496527ac5cbd405c9833c54b00d0529298981cbd3c37bfcec9c078c7f49f5cafca1834f95226c056e9426a8c8a714f4411e34356b77315469089d13011d4576b25ec9af7904c5a4f1c9ee0017a23a3a3a88c5ec3abb8616c4db43584af61bdacc310b163affb7691536a264130b813a00e0b670087c2d2a154ce09a01b8618272729616b1bbc33102947b28b95b4e5ad8c6a75bc72795c1b70755245245d0495c6f58191088435ee83b5a162e9b476ed3335253803fdf2c06f9029ecd7b6c062aaedda44e0a4541611631a2

Chave compartilhada decapsulada (hex): 8641fbf52902ea960c1398d46b57e9a5545f9665e74d7d8e676a8a7d0329e100
=== Descriptografia ===
Chave compartilhada (hex): 8641fbf52902ea960c1398d46b57e9a5545f9665e74d7d8e676a8a7d0329e100
Nonce (hex): 24e6f94dcbb2e859d72c4f598104de0c
Ciphertext (hex): 5311179413
Tag (hex): a675fae5497eeed5e574ec6d59e1cf79
Erro na descriptografia (MAC check failed): MAC check failed


ValueError: MAC check failed